In [1]:
# tensorflow imports
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, Dense, Input, Activation, AveragePooling2D, Flatten, ZeroPadding2D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAvgPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import RandomNormal, GlorotNormal
from tensorflow.keras.utils import plot_model
from scipy.ndimage.interpolation import rotate
from scipy import signal
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import CSVLogger

from tqdm import tqdm
import numpy as np
import os
import pywt
from matplotlib import pyplot as plt
from pywt._doc_utils import wavedec2_keys, draw_2d_wp_basis
import datetime
import warnings
warnings.filterwarnings("ignore")

In [4]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')


In [2]:
# a filter
filter1 = np.array([[[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [-1,-2,-3], [1,2,3]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]]])
filtera1 = rotate(filter1, angle=90)
filtera2 = rotate(filter1, angle=180)
filtera3 = rotate(filter1, angle=270)


# d filter
filter2 = np.array([[[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [-1,-2,-3],[2,4,6],[-1,-2,-3], [0,0,0]],
                    [[0,0,0], [2,4,6],[-4,-8,-12], [2,4,6],[0,0,0]],
                    [[0,0,0], [-1,-2,-3], [2,4,6], [-1,-2,-3], [0,0,0]],
                    [[0,0,0],[0,0,0],[0,0,0], [0,0,0], [0,0,0]]])


# b filter
fitler3 = np.array([[[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [1,2,3], [-2,-4,-6], [1,2,3], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]]])
filterb1 =  signal.convolve(filter2, filter2, mode='same')
filterb2 =  signal.convolve(filter2, filter2, mode='same')
filterb3 =  signal.convolve(filter2, filter2, mode='same')

# c filter
filter4 = np.array([[[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]], [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]], [
    [0,0,0], [1,2,3],[-3,-6,-9],[3,6,9],[-1,-2,-3]], [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                    [[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]]])

#this is not filter
filter4RDT = np.array([[[0,0,0], [0,0,0], [0,0,0], [0,0,0], [0,0,0]],
                       [[0,0,0], [0,0,0], [1,2,3], [0,0,0], [0,0,0]],
                       [[0,0,0], [0,0,0], [-3,-6,-9], [0,0,0], [0,0,0]],
                       [[0,0,0], [0,0,0], [3,6,9], [0,0,0], [0,0,0]],
                       [[0,0,0], [0,0,0], [-1,-2,-3], [0,0,0], [0,0,0]]])


filterc1 =  signal.convolve(filter4, filter4, mode='same')
filterc2 =  signal.convolve(filter4.T, filter4.T, mode='same').reshape(5,5,3)
filterc3 =  signal.convolve(filter4RDT, filter4RDT, mode='same').reshape(5,5,3)

#e filter
filter5 = np.array([[[-1,-2,-3], [2,4,6], [-2,-4,-6], [2,4,6], [-1,-2,-3]],
                    [[2,4,6], [-6,-12,-18], [-8,-16,-24], [-6,-12,-18], [2,4,6]],
                   [[-2,-4,-6], [8,16,24], [-12,-24,-36], [8,16,24], [2,4,6]],
                    [[2,4,6], [-6,-12,-18], [8,16,24], [-6,-12,-18], [2,4,6]],
                    [[-1,-2,-3], [2,4,6], [-2,-4,-6], [2,4,6], [-1,-2,-3]]])

filter_list = [filter1.tolist(), filtera1.tolist(), filtera2.tolist(), filtera3.tolist(), filter2.tolist(),\
               fitler3.tolist(), filterb1.tolist(), filterb2.tolist(), filterb3.tolist(),\
               filter4.tolist(), filterc1.tolist(), filterc2.tolist(), filterc3.tolist(), filter5.tolist()]

In [3]:
np.array([filter1,filtera1, filtera2, filtera3, filter2, fitler3, filterb1, filterb2, filterb3, filter4, filterc1]).shape

(11, 5, 5, 3)

In [4]:
%load_ext tensorboard

In [5]:
Inp = Input(shape = (64, 64, 3))
x = Conv2D(14, (5,5), padding = 'same', name = 'conv2d_1')(Inp)
x = Conv2D(16, (5,5), kernel_initializer= tf.keras.initializers.glorot_normal(seed=26), name = 'conv2d_2')(x)
x = ZeroPadding2D((1,1))(x)
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(32, (5,5), padding = 'same',kernel_initializer= tf.keras.initializers.glorot_normal(seed=26), name = 'conv2d_3')(x)
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(64, (3,3), padding = 'same',kernel_initializer= tf.keras.initializers.glorot_normal(seed=26), name = 'conv2d_4')(x)
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(128, (1,1), padding = 'same',kernel_initializer= tf.keras.initializers.glorot_normal(seed=26), name = 'conv2d_5')(x)
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(256, (1,1), padding = 'same',kernel_initializer= tf.keras.initializers.glorot_normal(seed=26), name = 'conv2d_6')(x)
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = AveragePooling2D((2,2))(x)
x = Flatten()(x)
out = Dense(1, activation = 'sigmoid',kernel_initializer= tf.keras.initializers.glorot_normal(seed=26), name = 'ouput')(x)

model = Model(inputs = [Inp], outputs = [out])
model.summary()

#plot_model(model, show_shapes =True)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 14)        1064      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 16)        5616      
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 62, 62, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 62, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 62, 62, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 16)        0     

In [6]:
try:
    os.mkdir('../models/logs')
except Exception as e:
    print(e)

[Errno 17] File exists: '../models/logs'


In [7]:
try:
    os.mkdir('../models/weights')
except Exception as e:
    print(e)

[Errno 17] File exists: '../models/weights'


In [8]:
csv_logger = CSVLogger("tanh_history_log.csv", append=True)

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
log_dir = "../models/logs" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
filepath = '../models/weights/model-tanh-try1.h5';
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_images=True)

In [10]:
earlyStopping = EarlyStopping(
    monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [11]:
layer = model.get_layer('conv2d_1')
layer.set_weights([np.array(filter_list).astype(np.float32).reshape(5,5,3,14), np.zeros(14,)])

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255.0,shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1/255.0)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '../input/patches/train/',  # This is the source directory for training images
        target_size=(64, 64),  # All images will be resized to 64x64
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        '../input/patches/validation/',  # This is the source directory for validation images
        target_size=(64, 64),  # All images will be resized to 64x64
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 1573068 images belonging to 2 classes.
Found 219603 images belonging to 2 classes.


In [13]:
history = model.fit(
      train_generator,  
      epochs=1,
    validation_data = validation_generator,
      verbose=1,
      callbacks=[tensorboard_callback, checkpoint, earlyStopping, csv_logger])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 49159 steps, validate for 6863 steps
    1/49159 [..............................] - ETA: 22:13:26

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/conv2d_1/Conv2D (defined at <ipython-input-13-fb7908d20afc>:6) ]] [Op:__inference_distributed_function_2370]

Function call stack:
distributed_function
